<img src="https://mikkolad.github.io/lundpython/front.jpeg" width="1400">

<h1><center> Where to get the lecture notes </center></h1>

You can get copies of all the lecture files at my personal website. Go to:

http://www.astro.lu.se/~mikkola/

and click on **Teaching**


Each lecture contains (as notebooks)
- Manual 
- Exercises (not present in lesson 4)
- Presentation

---

# PEP8

Recall from lecture 1 [The Python Style Guide](https://www.python.org/dev/peps/pep-0008/) a.k.a. PEP8.  

It is a lengthy document that can be hard to memorize. Instead, there are nifty tools one can use to check the PEP8 compliance of a script. Consider for example [pycodestyle](https://pypi.org/project/pycodestyle/). Once it has been installed on your system, you can check a script in the following way:

In [ ]:
import os
os.system('open -a Terminal .')  # Opens a terminal on MacOS

If you're working in a Jupyter notebook, you can use the magic command called [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) instead. A demonstration:

In [ ]:
%load_ext pycodestyle_magic
# Using lines longer than 80 characters within a notebook could be reasonable
%flake8_on --max_line_length 99

In [ ]:
a='This code is not PEP 8 compliant! Not only will pycodestyle get very upset, it will make sure you will be upset too.' 
for sentence in a.split( '! ' ):
  print(sentence ,end ='\n\n')# Notice how the Python interpreter does not require 4 space indents

# Docstrings

Docstrings contain documentation information for different functions in Python and we have a few ways of accessing them. But first, let's write our own docstring.

In [ ]:
def foo():
    """This is a docstring for foo(), it does nothing as a function"""
    return

As specified, our function foo() does nothing. But how would we know if we didn't write it ourselves? 

In [ ]:
help(foo)

With the `help()` function we can access the docstring, which can give us useful information on what a function does. We want to write docstrings if we work with other people.

Let's also access the docstring of some existing function!

In [ ]:
from numpy import identity
help(identity)

Jupyter notebook also has some useful ways of accessing docstrings. We can use<code style="color:#AA29FF"><b>?</b></code> and <code style="color:#AA29FF"><b>??</b></code> for example to access the docstring and source code respectively

In [ ]:
identity?

In [ ]:
identity??

We can also utilize `Shift + Tab` inside a function.  
1 `Tab` brings up a brief docstring.  
2 `Tab` makes it bigger.  
3 `Tab` makes it linger for 10 seconds.  
4 `Tab` opens the pager.

In [ ]:
identity()

# Peformance optimization & profiling

You might find that code you've written runs very slowly. In order to identify what the source of your slowness is you'll want to use profilers.  

You've already encountered [timeit](https://docs.python.org/3/library/timeit.html) so let's go over some more extensive alternatives.  

First, let's create a function that we want to profile:

In [ ]:
%load_ext heat
%load_ext line_profiler
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def movmean(xdata, ydata, window):
    ydata_new = np.zeros(len(ydata))
    xdata_new = np.zeros(len(xdata))
    k = int(window/2)
    for i in range(len(ydata)):
        if i < window:
            ydata_new[i] = np.mean(ydata[:(i+k)])
            xdata_new[i] = np.mean(xdata[:(i+k)])
        elif i > len(ydata)-window:
            ydata_new[i] = np.mean(ydata[(i-k):])
            xdata_new[i] = np.mean(xdata[(i-k):])
        else:
            ydata_new[i] = np.mean(ydata[(i-k):(i+k)])
            xdata_new[i] = np.mean(xdata[(i-k):(i+k)])
    return(xdata_new, ydata_new)

This is a [moving average](https://en.wikipedia.org/wiki/Moving_average).

We'll need some data to use it on:

In [ ]:
x, y = np.loadtxt('xy.txt')
x_med, y_med = movmean(x, y, 100)

plt.plot(x, y, '.')
plt.plot(x_med, y_med, 'r')
plt.xlabel('$x$')
plt.ylabel('$y$')

In Jupyter we can profile it with this [line_profiler](https://github.com/pyutils/line_profiler)

In [ ]:
%lprun -f movmean movmean(x, y, 100)

or we can profile it with [pyheat](https://github.com/csurfer/pyheatmagic)

In [ ]:
%%heat
import numpy as np

def movmean(xdata, ydata, window):
    ydata_new = np.zeros(len(ydata))
    xdata_new = np.zeros(len(xdata))
    k = int(window/2)
    for i in range(len(ydata)):
        if i < window:
            ydata_new[i] = np.mean(ydata[:(i+k)])
            xdata_new[i] = np.mean(xdata[:(i+k)])
        elif i > len(ydata)-window:
            ydata_new[i] = np.mean(ydata[(i-k):])
            xdata_new[i] = np.mean(xdata[(i-k):])
        else:
            ydata_new[i] = np.mean(ydata[(i-k):(i+k)])
            xdata_new[i] = np.mean(xdata[(i-k):(i+k)])
    return(xdata_new, ydata_new)


x, y = np.loadtxt('xy.txt')
x_med, y_med = movmean(x, y, 100)

# Spyder
Spyder uses `line_profiler` too in the package [spyder-line-profiler](https://github.com/spyder-ide/spyder-line-profiler)

A quick demonstration!

In [ ]:
import os
os.popen('spyder')

# Command line
When profiling on the command line, I again encourage you to use `line_profiler`. 

We will need the command: `kernprof -l -v spyderexample.py`

In [ ]:
import os
os.system('open -a Terminal .')  # Opens a terminal on MacOS

<h1><center> RISE </center></h1>